# Interactive chat!

I wanted to make the recommendation system a bit more user-friendly, so I wanted to create a "chatbot" by utilizing the template so that based on the keywords, it would generate the recommendation along with some additional features. 

I am also applying this functionality to the entire dataset, as I want a bit more of a free range for this part. 

The basis of my code came from the following page: [Use Tkinter To Design GUI Layout](https://www.pythonguis.com/tutorials/use-tkinter-to-design-gui-layout/) (Willman, J, 2024).

***Please note that if you run this entire notebook at once, two windows will pop up immediately. My suggestion is to go cell by cell to see the difference between the two "chats."***

***If nothing is popping up after the keyword search in the pop-up window, close it, and please run the notebook again.***

In [1]:
#Import the necessary libraries 
import pandas as pd 
import numpy as np 
from tkinter import *

from PIL import Image, ImageTk
import urllib.request
import io
import threading

np.set_printoptions(threshold=np.inf)

Click play on the cell below to have a pop up window of the interactive chat! Try some keywords such as retro," wedding," "vibrant," or a combination to explore suggestions! Make sure they are comma-separated, though. 

In [4]:
# Load the df 
film = pd.read_csv('../PLI18542171_Personalization_Final Project/Data/filmtypes.csv')

def generate_film_recommendations(film, keywords):
    # Identifying all of the necessary columns
    required_columns = {'Description', 'Film type', 'Film grain', 'Film contrast', 'Film hue', 'Film use cases'}
    
    # Used LLM to finetune these few blocks so that the output would function
    # Check for all required columns are present in the df
    missing_columns = required_columns - set(film.columns)
    if missing_columns:
        # Print a message if any required columns are missing
        print(f"Missing columns from the DataFrame: {', '.join(missing_columns)}")
        return pd.DataFrame()  
    
    # Create a new column 'all_text' by concatenating all relevant columns
    film['all_text'] = film[list(required_columns - {'Film use cases'})].fillna('').agg(' '.join, axis=1)
    
    # Prepare keywords by stripping whitespace and splitting by commas
    keywords = [keyword.strip() for keyword in keywords.split(',')]
    
    # Create a condition to filter films based on the presence of keywords in 'all_text'
    condition = film['all_text'].str.contains('|'.join(keywords), case=False, na=False)
    
    # Filter the df
    filtered_film = film[condition]
    return filtered_film

def fetch_and_display_results():
    # Get keywords from the user input in the GUI
    keywords = entry.get()
    
    # Generate film recommendations based on the keywords
    recommended_films = generate_film_recommendations(film, keywords)
    
    # Used LLM to finetune this
    # Enable the text widget
    text_widget.config(state=NORMAL)
    text_widget.delete('1.0', END)
    
    # Used LLM to finetune this
    # Check if the recommended films DataFrame is not empty
    if not recommended_films.empty:
        # Iterate through each row in the DataFrame
        for _, row in recommended_films.iterrows():
            # Start building the film information string with the film name and description
            film_info = f"{row['Film name']}: {row['Description']}"
            
            # Append additional information if available
            additional_info = " | ".join(f"{key}: {row[key]}" for key in ['Film type', 'Film grain', 'Film contrast', 'Film hue', 'Film use cases'] if pd.notna(row[key]))
            text_widget.insert(END, f"{film_info} | {additional_info}\n\n")
    else:
        # Display a message if no films were found
        text_widget.insert(END, "No films found with those keywords.\n")
    
    # Disable the text widget to prevent user editing
    text_widget.config(state=DISABLED)

# Create the main window for the application
root = Tk()
root.title("Film Recommendations Display")
root.config(bg="skyblue")

# Create a frame for input elements
left_frame = Frame(root, width=200, height=400)
left_frame.grid(row=0, column=0, padx=10, pady=5)

# Add a label and entry widget for keyword input
Label(left_frame, text="Enter Keywords (comma-separated):").grid(row=1, column=0, padx=5, pady=5)
entry = Entry(left_frame, width=20)
entry.grid(row=2, column=0, padx=5, pady=5)

# Create a text widget for displaying results
text_widget = Text(root, height=20, width=50)
text_widget.grid(row=0, column=1, padx=10, pady=5)
text_widget.config(state=DISABLED)

# Add a button that triggers the result display function
button = Button(left_frame, text="Get Results", command=fetch_and_display_results)
button.grid(row=3, column=0, padx=5, pady=5)

# Start the GUI event loop
root.mainloop()

# Chat with Photo Displays!

Here I also started to workout a code function so that it would also produce the corresponding image based on the keyword output. However, there is still some glitches if too many results are generated, so the images are a bit distorted. 

If you have a specific inquiry, like "Kodak Portra 400", it should display the textual information and the corresponding image correctly. Otherwise, it bugs out from time to time.

The addition of the photo code was done on the basis of this site: [How to place an image into a frame in Tkinter](https://www.tutorialspoint.com/how-to-place-an-image-into-a-frame-in-tkinter) (Tutorials Point, no date).

***If nothing is being generated up after the keyword search in the pop-up window, close it, and please run the notebook again.***

In [3]:
# Load the DataFrame
film = pd.read_csv('../PLI18542171_Personalization_Final Project/Data/filmtypes.csv')

def display_image_from_url(url, image_frame):
    # Attempt to load and display an image from a URL
    # Used LLM to debug this
    try:
        with urllib.request.urlopen(url) as u:
            raw_data = u.read()
        image = Image.open(io.BytesIO(raw_data))
        photo = ImageTk.PhotoImage(image)
        label = Label(image_frame, image=photo)
        label.image = photo  # Keep a reference to avoid garbage collection
        label.pack()
    except Exception as e:
        print(f"Failed to load image from {url}. Error: {e}")

def generate_film_recommendations(film, keywords):
    # Define the required columns
    # Used LLM to debug this
    required_columns = {'Description', 'Film type', 'Film grain', 'Film contrast', 'Film hue', 'Film use cases', 'Photo of the film'}
    if not required_columns.issubset(film.columns):
        print("One or more required columns are missing from the DataFrame.")
        return pd.DataFrame()
    
    # Concatenate text from all relevant columns to create a searchable text column
    film['all_text'] = film[list(required_columns - {'Photo of the film'})].fillna('').agg(' '.join, axis=1)
    
    keywords = [keyword.strip() for keyword in keywords.split(',')]
    condition = film['all_text'].str.contains('|'.join(keywords), case=False, na=False)
    filtered_film = film[condition]
    return filtered_film

# Used LLM to debug this
def fetch_and_display_results(image_frame):
    # Fetch keywords from the GUI entry widget
    keywords = entry.get()
    recommended_films = generate_film_recommendations(film, keywords)
    text_widget.config(state=NORMAL)
    text_widget.delete('1.0', END)
    
    # Display each recommended film and its image if available
    if not recommended_films.empty:
        for _, row in recommended_films.iterrows():
            film_info = f"{row['Film name']}: {row['Description']}"
            text_widget.insert(END, film_info + "\n")
            if pd.notna(row['Photo of the film']):
                threading.Thread(target=display_image_from_url, args=(row['Photo of the film'], image_frame)).start()
    else:
        text_widget.insert(END, "No films found with those keywords.\n")
    
    text_widget.config(state=DISABLED)

# Set up the main window
root = Tk()
root.title("Film Recommendations Display with photos")
root.config(bg="skyblue")

# Set up a frame for input elements
left_frame = Frame(root, width=200, height=400)
left_frame.grid(row=0, column=0, padx=10, pady=5)

# Add a label and entry widget for keyword input
Label(left_frame, text="Enter Keywords (comma-separated):").grid(row=1, column=0, padx=5, pady=5)
entry = Entry(left_frame, width=20)
entry.grid(row=2, column=0, padx=5, pady=5)

# Set up a text widget for displaying film information
text_widget = Text(root, height=20, width=50)
text_widget.grid(row=0, column=1, padx=10, pady=5)
text_widget.config(state=DISABLED)

# Set up a frame for displaying images
image_frame = Frame(root, width=1000, height=1000)
image_frame.grid(row=0, column=2, padx=10, pady=5)

# Add a button that triggers the result display function in a new thread
button = Button(left_frame, text="Get Results", command=lambda: threading.Thread(target=fetch_and_display_results, args=(image_frame,)).start())
button.grid(row=3, column=0, padx=5, pady=5)

# Start the GUI event loop
root.mainloop()

### **References**: 
Willman, J. (2024) - Use Tkinter To Design GUI Layout, Pythonguis. Available at: https://www.pythonguis.com/tutorials/use-tkinter-to-design-gui-layout/ (Accessed: 8 June 2024).

Tutorials Point (no date) - How to place an image into a frame in Tkinter? Available at: https://www.tutorialspoint.com/how-to-place-an-image-into-a-frame-in-tkinter (Accessed: 8 June 2024). 

### **LLM Disclaimer**

LLM was partially used throughout the project in order to de-bug code issues that arose with the notebook. You can see additional specific LLM Disclaimers throughout the notebook.